# Day 22: AWS Glue & Data Catalog - Interactive Exploration

This notebook provides interactive exploration of AWS Glue and Data Catalog capabilities using LocalStack.


In [ ]:
# Setup and imports
import boto3
import pandas as pd
import os
from dotenv import load_dotenv

# Load environment
load_dotenv()

# Initialize AWS clients for LocalStack
endpoint_url = 'http://localstack:4566'

s3_client = boto3.client('s3', endpoint_url=endpoint_url)
glue_client = boto3.client('glue', endpoint_url=endpoint_url)
athena_client = boto3.client('athena', endpoint_url=endpoint_url)

print("✅ AWS clients initialized for LocalStack")

## 1. Explore Data Catalog


In [ ]:
# List Glue databases
try:
    databases = glue_client.get_databases()
    print(f"📊 Found {len(databases['DatabaseList'])} databases:")
    for db in databases['DatabaseList']:
        print(f"   • {db['Name']}: {db.get('Description', 'No description')}")
except Exception as e:
    print(f"ℹ️  No databases found or error: {e}")

In [ ]:
# List tables in database
database_name = 'serverlessdata_analytics'

try:
    tables = glue_client.get_tables(DatabaseName=database_name)
    print(f"📋 Found {len(tables['TableList'])} tables in {database_name}:")
    for table in tables['TableList']:
        print(f"   • {table['Name']}: {len(table['StorageDescriptor']['Columns'])} columns")
except Exception as e:
    print(f"ℹ️  No tables found or error: {e}")

## 2. Explore S3 Data


In [ ]:
# List S3 buckets
try:
    buckets = s3_client.list_buckets()
    print(f"🪣 Found {len(buckets['Buckets'])} S3 buckets:")
    for bucket in buckets['Buckets']:
        print(f"   • {bucket['Name']}")
except Exception as e:
    print(f"❌ Error listing buckets: {e}")

In [ ]:
# Explore data structure
bucket_name = 'serverlessdata-datalake'

try:
    objects = s3_client.list_objects_v2(Bucket=bucket_name, Prefix='raw/', Delimiter='/')
    
    if 'CommonPrefixes' in objects:
        print(f"📁 Data folders in {bucket_name}/raw/:")
        for prefix in objects['CommonPrefixes']:
            folder = prefix['Prefix'].rstrip('/')
            print(f"   • {folder}")
    
    if 'Contents' in objects:
        print(f"\n📄 Files in root:")
        for obj in objects['Contents'][:5]:  # Show first 5 files
            print(f"   • {obj['Key']} ({obj['Size']} bytes)")
            
except Exception as e:
    print(f"ℹ️  No objects found or error: {e}")

## 3. Sample Data Analysis


In [ ]:
# Download and analyze sample transaction data
try:
    # List transaction files
    objects = s3_client.list_objects_v2(
        Bucket=bucket_name, 
        Prefix='raw/transactions/',
        MaxKeys=1
    )
    
    if 'Contents' in objects and len(objects['Contents']) > 0:
        # Get first transaction file
        first_file = objects['Contents'][0]['Key']
        print(f"📊 Analyzing sample file: {first_file}")
        
        # Download and read CSV
        response = s3_client.get_object(Bucket=bucket_name, Key=first_file)
        df = pd.read_csv(response['Body'])
        
        print(f"\n📈 Data Summary:")
        print(f"   • Records: {len(df):,}")
        print(f"   • Columns: {len(df.columns)}")
        print(f"   • Date range: {df['transaction_date'].min()} to {df['transaction_date'].max()}")
        print(f"   • Total amount: ${df['transaction_amount'].sum():,.2f}")
        
        # Show sample records
        print(f"\n📋 Sample Records:")
        display(df.head())
        
        # Basic analytics
        print(f"\n📊 Quick Analytics:")
        print(f"   • Average transaction: ${df['transaction_amount'].mean():.2f}")
        print(f"   • Unique customers: {df['customer_id'].nunique():,}")
        print(f"   • Payment methods: {df['payment_method'].nunique()}")
        
    else:
        print("ℹ️  No transaction files found")
        
except Exception as e:
    print(f"❌ Error analyzing data: {e}")

## 4. Glue Crawler Simulation


In [ ]:
# List existing crawlers
try:
    crawlers = glue_client.get_crawlers()
    print(f"🕷️ Found {len(crawlers['Crawlers'])} crawlers:")
    
    for crawler in crawlers['Crawlers']:
        print(f"\n   • {crawler['Name']}")
        print(f"     State: {crawler['State']}")
        print(f"     Database: {crawler['DatabaseName']}")
        
        if 'LastCrawl' in crawler and crawler['LastCrawl']:
            last_crawl = crawler['LastCrawl']
            print(f"     Last crawl: {last_crawl.get('Status', 'Unknown')}")
            
except Exception as e:
    print(f"ℹ️  No crawlers found or error: {e}")

## 5. ETL Job Exploration


In [ ]:
# List Glue jobs
try:
    jobs = glue_client.get_jobs()
    print(f"⚙️ Found {len(jobs['Jobs'])} ETL jobs:")
    
    for job in jobs['Jobs']:
        print(f"\n   • {job['Name']}")
        print(f"     Description: {job.get('Description', 'No description')}")
        print(f"     Max capacity: {job.get('MaxCapacity', 'Not set')} DPUs")
        print(f"     Timeout: {job.get('Timeout', 'Not set')} minutes")
        
except Exception as e:
    print(f"ℹ️  No jobs found or error: {e}")

## 6. Customer Analytics Simulation


In [ ]:
# Simulate customer analytics (what Glue ETL would do)
try:
    # Get sample transaction data
    objects = s3_client.list_objects_v2(
        Bucket=bucket_name, 
        Prefix='raw/transactions/',
        MaxKeys=3
    )
    
    if 'Contents' in objects:
        # Combine multiple files
        all_transactions = []
        
        for obj in objects['Contents']:
            response = s3_client.get_object(Bucket=bucket_name, Key=obj['Key'])
            df = pd.read_csv(response['Body'])
            all_transactions.append(df)
        
        # Combine all data
        combined_df = pd.concat(all_transactions, ignore_index=True)
        
        print(f"📊 Customer Analytics (Simulated ETL Output):")
        print(f"   Total transactions analyzed: {len(combined_df):,}")
        
        # Customer segmentation
        customer_summary = combined_df.groupby('customer_id').agg({
            'transaction_amount': ['sum', 'mean', 'count']
        }).round(2)
        
        customer_summary.columns = ['total_spent', 'avg_amount', 'transaction_count']
        customer_summary = customer_summary.reset_index()
        
        # Add customer segments
        customer_summary['segment'] = pd.cut(
            customer_summary['total_spent'],
            bins=[0, 500, 2000, float('inf')],
            labels=['basic', 'standard', 'premium']
        )
        
        print(f"\n🎯 Customer Segmentation:")
        segment_summary = customer_summary['segment'].value_counts()
        for segment, count in segment_summary.items():
            print(f"   • {segment}: {count:,} customers")
        
        print(f"\n💰 Revenue by Segment:")
        revenue_by_segment = customer_summary.groupby('segment')['total_spent'].sum()
        for segment, revenue in revenue_by_segment.items():
            print(f"   • {segment}: ${revenue:,.2f}")
        
        # Show top customers
        print(f"\n🏆 Top 5 Customers:")
        top_customers = customer_summary.nlargest(5, 'total_spent')
        display(top_customers)
        
except Exception as e:
    print(f"❌ Error in analytics: {e}")

## 7. Cost Optimization Analysis


In [ ]:
# Analyze data for cost optimization opportunities
print("💰 Cost Optimization Analysis:")

try:
    # Analyze file sizes and formats
    total_size = 0
    file_count = 0
    
    objects = s3_client.list_objects_v2(Bucket=bucket_name, Prefix='raw/')
    
    if 'Contents' in objects:
        for obj in objects['Contents']:
            total_size += obj['Size']
            file_count += 1
    
    print(f"\n📊 Current Data Stats:")
    print(f"   • Total files: {file_count:,}")
    print(f"   • Total size: {total_size / (1024*1024):.2f} MB")
    print(f"   • Average file size: {(total_size / file_count) / 1024:.2f} KB")
    
    print(f"\n💡 Optimization Recommendations:")
    print(f"   • Convert CSV to Parquet: ~70% size reduction")
    print(f"   • Apply compression: Additional ~50% reduction")
    print(f"   • Partition by date: Faster queries, lower costs")
    print(f"   • Columnar format: Better for analytics")
    
    # Estimate cost savings
    current_storage_cost = (total_size / (1024**3)) * 0.023  # $0.023 per GB/month
    optimized_storage_cost = current_storage_cost * 0.15  # 85% reduction
    monthly_savings = current_storage_cost - optimized_storage_cost
    
    print(f"\n💰 Estimated Monthly Costs:")
    print(f"   • Current (CSV): ${current_storage_cost:.4f}")
    print(f"   • Optimized (Parquet): ${optimized_storage_cost:.4f}")
    print(f"   • Monthly savings: ${monthly_savings:.4f}")
    
except Exception as e:
    print(f"❌ Error in cost analysis: {e}")

## 8. Next Steps

This notebook demonstrated:
- ✅ Data Catalog exploration
- ✅ S3 data analysis
- ✅ Customer analytics simulation
- ✅ Cost optimization analysis

**Try these next:**
1. Run the complete demo: `python demo.py`
2. Explore the exercise: `python exercise.py`
3. Check the solution: `python solution.py`
4. Take the quiz: Review `quiz.md`
